## Store Numpy Arrays in a SQLite Database

In [16]:
import os
import io
import sqlite3
import numpy as np
import pandas as pd

#### Load the P3B3 Synthetic Data 

In [2]:
!ls ../data/numpy/P3B3/processed/

test_data.npy  test_labels.npy	train_data.npy	train_labels.npy


In [3]:
ARRY_PATH = '../data/numpy/P3B3/processed/'

In [27]:
def load_data(path, trainset=True):
    """ Load the P3B3 array data """
    if trainset:
        data = np.load(os.path.join(path, 'train_data.npy'))
        label = np.load(os.path.join(path, 'train_labels.npy'))
    else:
        data = np.load(os.path.join(path, 'test_data.npy'))
        label = np.load(os.path.join(path, 'test_labels.npy'))
        
    return (data, label)

In [28]:
def batch_arrays(x_train, y_train, split='train'):
    """ Create a list of tuples, each tuple being a single sample of data """
    assert x_train.shape[0] == y_train.shape[0]
    num_samples = x_train.shape[0]
    return [(x_train[i], y_train[i], split) for i in range(num_samples)]

In [45]:
x_train, y_train = load_data(ARRY_PATH)
x_test, y_test = load_data(ARRY_PATH, False)

train_data = batch_arrays(x_train, y_train, 'train')
test_data = batch_arrays(x_test, y_test, 'test')

In [31]:
dataset = train_data + test_data

#### Helper functions to store nd.arrays into SQLite

In [35]:
def adapt_array(arry):
    """ Convert numpy array to binary string
    
    References
    ----------
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arry)
    out.seek(0)
    return sqlite3.Binary(out.read())

In [36]:
def convert_array(text):
    """ Convert text to numpy array """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [37]:
# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

#### Create the database

In [38]:
conn = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

In [39]:
with conn:
    cur.execute("""
        CREATE TABLE trainset (
            data array,
            label array,
            split text
        )"""
    )

In [40]:
with conn:
    cur.executemany("INSERT INTO trainset VALUES (?,?,?)", dataset)

#### Fetch the first sample of data with label corresponding labels

In [41]:
with conn:
    cur.execute("SELECT * from trainset")
    
data, label, split = cur.fetchone()

In [42]:
data

array([ 35, 197, 232, ...,   0,   0,   0])

In [43]:
label

array([5, 0, 1, 2])

In [44]:
split

'train'